<a href="https://colab.research.google.com/github/K-RLange/relatio/blob/master/execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! python -m pip install -U pip wheel setuptools

# install the package
!pip install -U git+https://github.com/K-RLange/relatio.git
#! python -m pip install -U relatio
! python -m pip install -U pandas
! python -m pip install --upgrade google-cloud-storage

# download SpaCy and NLTK additional resources
! python -m spacy download en_core_web_sm
! python -m nltk.downloader punkt wordnet stopwords averaged_perceptron_tagger
import pickle
from relatio.utils import split_into_sentences
from relatio.wrappers import run_srl
import pandas as pd
import nltk
from tqdm import tqdm

nltk.download("stopwords")
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import tqdm
from relatio.wrappers import get_narratives
import spacy
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
from relatio.wrappers import build_narrative_model
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/My Drive/Relatio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 31.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/K-RLange/relatio.git to /tmp/pip-req-build-4oxzoioy
  Running command git cl

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


Mounted at /content/gdrive
/content/gdrive/My Drive/Relatio


In [ ]:
data = pd.read_csv("InflationParagraphs6.csv", encoding="unicode_escape").x#pickle.load(open("nyt1.pickle", "rb"))
sub_data = data#[x[0] for x in data[0:50]]# if "inflation" in x or "Inflation" in x[0]]
df = pd.DataFrame({"id": [x for x in range(len(sub_data))], "doc": [x for x in sub_data]})

In [ ]:
split_sentences = split_into_sentences(
    df, progress_bar=True
)
srl_res = run_srl(
    path="https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",  # pre-trained model
    sentences=split_sentences[1],
    progress_bar=True,
)

Splitting into sentences...


100%|██████████| 20100/20100 [12:28<00:00, 26.84it/s]


Running SRL...


100%|██████████| 1645/1645 [4:58:21<00:00, 10.88s/it]


In [ ]:
#list(map(len, srl_res))
pickle.dump(split_sentences, open("sentences.pickle", "wb"))
pickle.dump(srl_res, open("srl.pickle", "wb"))

In [ ]:
split_sentences = pickle.load(open("sentences.pickle", "rb"))
srl_res = pickle.load(open("srl.pickle", "rb"))

In [ ]:
srl_res[1429]

In [ ]:
narrative_model = build_narrative_model(
    srl_res=srl_res,
    sentences=split_sentences[1],
    embeddings_type="gensim_keyed_vectors",  # see documentation for a list of supported types
    embeddings_path="glove-wiki-gigaword-300",
    n_clusters=[[2]],
    top_n_entities=100,
    stop_words=spacy_stopwords,
    remove_n_letter_words=1,
    progress_bar=True,
)
print(narrative_model.keys())
narrative_model['entities'].most_common()[:20]

Processing SRL...


100%|██████████| 237615/237615 [00:40<00:00, 5813.57it/s]


finished
Cleaning SRL...


100%|██████████| 976611/976611 [15:16<00:00, 1065.31it/s]


Computing role frequencies...


...

100%|██████████| 237615/237615 [29:47<00:00, 132.91it/s]


Mapping named entities...


100%|██████████| 976611/976611 [02:06<00:00, 7716.63it/s]


Loading embeddings model...
[==================================================] 100.0% 376.1/376.1MB downloaded
dict_keys(['roles_considered', 'roles_with_entities', 'roles_with_embeddings', 'dimension_reduce_verbs', 'clean_text_options', 'verb_counts', 'entities', 'top_n_entities', 'embeddings_model', 'cluster_model', 'cluster_labels_most_similar', 'cluster_labels_most_freq'])


[('fed', 22044),
 ('united state', 5899),
 ('china', 5326),
 ('american', 5293),
 ('european', 3028),
 ('federal reserve', 2486),
 ('republican', 2295),
 ('na', 2163),
 ('chinese', 2107),
 ('treasury', 1942),
 ('japan', 1782),
 ('congress', 1728),
 ('powell', 1669),
 ('germany', 1560),
 ('trump', 1513),
 ('bernanke', 1462),
 ('britain', 1359),
 ('washington', 1307),
 ('new york', 1240),
 ('democrat', 1197)]

In [ ]:
final_statements = get_narratives(
    srl_res=srl_res,
    doc_index=split_sentences[0],  # doc names
    narrative_model=narrative_model,
    n_clusters=[0],
    progress_bar=True,
)

Processing SRL...


100%|██████████| 237615/237615 [00:38<00:00, 6133.54it/s]


finished
Cleaning SRL...


100%|██████████| 976611/976611 [15:02<00:00, 1082.26it/s]


Processing raw arguments...


...

Mapping named entities...


100%|██████████| 976611/976611 [02:08<00:00, 7608.98it/s]


Assigning clusters to roles...


100%|██████████| 976611/976611 [05:05<00:00, 3199.06it/s]


test


In [ ]:
#list(map(len, srl_res))
pickle.dump(narrative_model, open("model.pickle", "wb"))
pickle.dump(final_statements, open("final_statements.pickle", "wb"))

In [ ]:
narrative_model = pickle.load(open("model.pickle", "rb"))
final_statements = pickle.load(open("final_statements.pickle", "rb"))

In [ ]:
print(final_statements.head())
def extend_inside(lists):
    new_list = []
    for element in lists:
        for part_of_element in element:
            new_list.append(part_of_element)
    return new_list

#Detect maximum number of statements within a sentence using a greedy-approach
#of finding the maximum number of disjoint statements. Starts with the statement
#that ends first of all statements
def max_disjoint_intervals(statements, doc, sentence, condition=["doc"]):
    indices = []
    list_ = [[statements["statement"][x], statements["start"][x], statements["end"][x]] for 
             x in range(len(statements)) if statements["doc"][x] == doc 
             and statements["sentence"][x] == sentence
             and all([statements[y][x] for y in condition])]
    if len(list_) > 0:
        # Lambda function to sort the list
        list_.sort(key = lambda x: x[2])
        indices.append(list_[0][0])

        # End point of first interval
        r1 = list_[0][2]
        
        for i in tqdm(range(1, len(list_))):
            l1 = list_[i][1]
            r2 = list_[i][2]

            if l1 > r1:
                indices.append(list_[i][0])
                r1 = r2
        final_statements = [statements.loc[x].to_dict() for x in range(len(statements)) if 
                            statements["statement"][x] in indices]
        return final_statements
    else:
        return None

filtered_statements = []
for doc in list(set(final_statements.doc)):
    for sentence in list(set(final_statements["sentence"][final_statements["doc"] == doc])):
        current_res = max_disjoint_intervals(final_statements, doc, sentence, condition=["ARG0_highdim", "B-V_highdim", "ARG1_highdim"])
        if current_res:
            filtered_statements.append(current_res) 

   doc  sentence  statement         ARG0_highdim ARG0_lowdim B-V_highdim  \
0    0         0          0                                         fell   
1    0         0          1  recent housing data   inflation    indicate   
2    0         0          2                                                
3    0         0          3                                        start   
4    0         0          4                                      decline   

  B-V_lowdim B-ARGM-NEG_highdim B-ARGM-NEG_lowdim B-ARGM-MOD_highdim  \
0       come                                                           
1    suggest                                                           
2                                                                      
3      begin                                                           
4    decline                                                           

          ARG1_highdim ARG1_lowdim ARG2_highdim ARG2_lowdim  start  end  
0  consumer confidence   inflation  

KeyboardInterrupt: ignored

In [ ]:
#print(filtered_statements[0])
print(current_res)
filtered_statements_df = pd.DataFrame(extend_inside(filtered_statements))
print(filtered_statements_df)
print(len(filtered_statements_df))

In [ ]:
RELATIOn_list = ["because", "thus", "therefore", "after", "then", "if", "hence", "consequently", "resulting",
                     "since", "reason", "due"]
def filter_for_RELATIOns(doc, RELATIOns=RELATIOn_list, sentences=srl_res):
    sentence = sentences[doc]["words"]
    appearance_dict = {}
    for word in RELATIOns:
        index = [i for i, x in enumerate(sentence) if x == word]
        if len(index) > 0:
            appearance_dict[word] = index
    return appearance_dict

RELATIOns = list(map(filter_for_RELATIOns, range(len(srl_res))))

doc = -1
sentence = -1
last_end =-1
statement_list = [[] for x in range(max(split_sentences[0]) + 1)]
for iloc in range(len(filtered_statements_df)):
    statement = filtered_statements_df.iloc[iloc]
    if statement.doc > doc:
        doc = statement.doc
        sentence = statement.sentence
        last_end = -1
        current_start = statement.start
    elif statement.sentence > sentence:
        current_start = 100000
    else:
        current_start = statement.start
    current_statement = []
    if len(RELATIOns[sentence]) > 0:
        for relation in RELATIOns[sentence].keys():
            if any([x >= last_end and x < current_start for x in RELATIOns[sentence][relation]]):
                current_statement.append(relation)
    current_statement.append(statement["ARG0_highdim"])
    current_statement.append(statement["B-V_highdim"])
    current_statement.append(statement["ARG1_highdim"])
    statement_list[doc].append(current_statement)
    last_end = statement.end
    sentence = statement.sentence
    
pickle.dump(statement_list, open("RELATIOn-statements.pickle", "wb"))